In [1]:
!pip install python-bidi

In [2]:
!pip install arabic-reshaper

# Import Libraries

In [3]:
import random
import json
import pickle
import numpy as np

import nltk
# english:
from nltk.stem import WordNetLemmatizer
# Arabic
from nltk.stem.isri import ISRIStemmer

from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model

import tkinter as tk
from tkinter import *
from bidi.algorithm import get_display
from arabic_reshaper import reshape

In [4]:
isstemmer = ISRIStemmer()

# ChatBot Inference Program Setup

In [5]:
data_file = open("intents_arabic.json", encoding='utf-8', errors= 'ignore').read()
intents = json.loads(data_file)

In [6]:
words= pickle.load(open('words.pkl', 'rb'))
classes= pickle.load(open('classes.pkl', 'rb'))

model = load_model('chatbot_model.h5')

In [7]:
words

['اذا',
 'ارد',
 'ارك',
 'استنساخك',
 'اسم',
 'اصطناعي؟',
 'امت',
 'امل',
 'ان',
 'انا',
 'انت',
 'انك',
 'بأن',
 'بأي',
 'باي',
 'بدو',
 'بضح',
 'بك',
 'بوت',
 'تحب',
 'تحر',
 'تسر',
 'تكون',
 'تمت',
 'ثرثار',
 'ثرثر',
 'جاب',
 'جدد',
 'جزل',
 'جسد',
 'جهز',
 'حاج',
 'حالك؟',
 'حجم',
 'حدث',
 'حذك',
 'حسن',
 'حكم',
 'حكيم؟',
 'خاص',
 'خالد؟',
 'خبر',
 'خدم',
 'خرج',
 'خصة',
 'خلد',
 'خلف',
 'خير',
 'دتا',
 'ذك؟',
 'ذكء',
 'ذلك',
 'رئع',
 'رائع،',
 'رحب',
 'رمج',
 'رنمج',
 'روبو',
 'سعد',
 'سلم',
 'سمح',
 'شعر',
 'شغل',
 'شكر',
 'شو',
 'صبح',
 'صحح',
 'صطناع',
 'صندوق',
 'ضيق',
 'عطل',
 'عل؟',
 'على',
 'علي',
 'عمل',
 'عنه',
 'غبة',
 'غبي',
 'غير',
 'فضل',
 'فعل',
 'فكر',
 'فهم',
 'في',
 'فيد',
 'فيه',
 'قتل',
 'قدم',
 'قهوة؟',
 'كتابتك؟',
 'كذب',
 'كل',
 'كمبيوتر',
 'كيف',
 'كين',
 'لا',
 'لحق',
 'لدك',
 'لغة',
 'لغي',
 'لك',
 'لها',
 'لي',
 'ما',
 'متى',
 'مثل',
 'محادثة؟',
 'مساء',
 'مشي',
 'مك؟',
 'من',
 'منتجك؟',
 'موت',
 'مين',
 'نت؟',
 'نظم',
 'نفس',
 'نوع',
 'نول',
 'هدف',
 'هذ

In [8]:
classes

['AI',
 'abbr',
 'artificial',
 'bend',
 'body',
 'bot1',
 'breathe',
 'business',
 'chatbot',
 'chatterbox',
 'clone',
 'comp',
 'computer',
 'control',
 'cramped',
 'date',
 'death',
 'do',
 'events',
 'fav',
 'fight',
 'goodbye',
 'greetings',
 'hardware',
 'hobby',
 'idea',
 'imortal',
 'lang',
 'laugh',
 'lie',
 'machine',
 'malfunction',
 'motormouth',
 'move',
 'name',
 'name1',
 'need',
 'noanswer',
 'os',
 'program',
 'programming',
 'ratchet',
 'robotics',
 'robots',
 'robotss',
 'sapient',
 'sense',
 'sentiment',
 'shoe',
 'sound',
 'stupid',
 'thanks',
 'usage',
 'who',
 'wt']

# Words Clean Up Bag of Words

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [isstemmer.stem(word) for word in sentence_words]
    #print("cleaned up sentence =", sentence_words)
    return sentence_words

In [10]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    
    for w in sentence_words:
        for i, word in enumerate(words):
            if word ==w:
                bag[i]=1
                
    return np.array(bag)

# ChatBot Testing

In [11]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    #print("row result",res)
    
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print("results after THRESHOLDING", results)
    
    return_list = []
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
        
    #print("final probabile results list", return_list)
    
    return return_list

In [12]:
def get_response(intents_list, intents_json):
    tag= intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [13]:
'''
while True:
    message = input("")
    ints = predict_class(message)
    #print("ints", ints)
    res = get_response(ints, intents)
    
    print(res)
'''

'\nwhile True:\n    message = input("")\n    ints = predict_class(message)\n    #print("ints", ints)\n    res = get_response(ints, intents)\n    \n    print(res)\n'

In [14]:
def chatbot_response(msg):
    
    ints = predict_class(msg)
    
    res = get_response(ints,intents)
    
    return res

In [15]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "أنت: " + msg + '\n\n', "right")

        res = chatbot_response(msg)
        #res_rtl = get_display(reshape(res), base_dir='R')
        ChatLog.insert(END, "الالي: " + res + '\n\n', "right")

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = tk.Tk()
base.title("مرحبا")
base.geometry("500x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                    command=send)

EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

# Placez les composants avec une direction RTL
scrollbar.place(x=6, y=6, height=386)  # Placez le scrollbar à gauche
ChatLog.place(x=40, y=6, height=386, width=440)  # Placez le ChatLog plus à droite
EntryBox.place(x=40, y=401, height=90, width=265)  # Placez l'EntryBox plus à droite
SendButton.place(x=6, y=401, height=90)  # Placez le bouton à gauche

base.mainloop()

1/1 [==============================] - 0s 27ms/step
